In [1]:
import pandas as pd
import weaviate
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np
from tqdm import tqdm


/home/nevin/anaconda3/envs/1/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Initialize Weaviate client
client = weaviate.Client("http://localhost:8080")


/tmp/ipykernel_433743/680488569.py:2: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client("http://localhost:8080")
/home/nevin/anaconda3/envs/1/lib/python3.11/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code t

In [5]:

# Load the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# Load the cross-encoder model
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Load the ground truth data
df_ground_truth = pd.read_csv('../data/ground-truth-retrieval.csv').fillna('')
queries = df_ground_truth['question'].tolist()
ground_truth_ids = df_ground_truth['q_id'].astype(str).tolist()

# Number of top results to consider
k = 10
rerank_size = 10  # Number of results to rerank

# Initialize metrics
hit_count = 0
reciprocal_ranks = []

for query_text, true_id in tqdm(zip(queries, ground_truth_ids), total=len(queries)):
    # Generate embedding for the query
    query_embedding = embedding_model.encode(query_text).astype('float32')

    # Perform initial hybrid search
    response = (
        client.query
        .get("Game", ["gameId", "gameName", "description"])
        .with_hybrid(
            query=query_text,
            alpha=0.5,
            vector=query_embedding.tolist()
        )
        .with_limit(rerank_size)
        .do()
    )

    # Extract the retrieved results
    results = response["data"]["Get"]["Game"]

    # Prepare pairs for reranking
    pairs = [(query_text, res['gameName'] + ' ' + res.get('description', '')) for res in results]

    # Get reranking scores
    rerank_scores = cross_encoder.predict(pairs)

    # Combine results with rerank scores
    for res, score in zip(results, rerank_scores):
        res['rerank_score'] = score

    # Sort results by rerank score
    reranked_results = sorted(results, key=lambda x: x['rerank_score'], reverse=True)[:k]
    retrieved_ids = [res['gameId'] for res in reranked_results]

    # Evaluate the results
    if true_id in retrieved_ids:
        hit_count += 1
        rank = retrieved_ids.index(true_id) + 1
        reciprocal_rank = 1 / rank
        reciprocal_ranks.append(reciprocal_rank)
    else:
        reciprocal_ranks.append(0)

# Calculate metrics
hit_rate = hit_count / len(queries)
mrr = np.mean(reciprocal_ranks)

print(f"Hit Rate@{k} after reranking: {hit_rate:.4f}")
print(f"MRR@{k} after reranking: {mrr:.4f}")


/home/nevin/anaconda3/envs/1/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 2950/2950 [24:51<00:00,  1.98it/s] 

Hit Rate@10 after reranking: 0.9715
MRR@10 after reranking: 0.8146
